### Sednoid Daten 
Daten holen von https://ssd.jpl.nasa.gov/ und in wikitable konvertieren
Verwendung für [Wikipedia-Vorlage: Sednoiden](https://de.wikipedia.org/wiki/Vorlage:Sednoiden)

Libs importieren

In [97]:
from urllib.request import urlopen
import json

JSON-Daten laden

In [98]:
url='https://ssd-api.jpl.nasa.gov/sbdb_query.api?fields=epoch.cal,full_name,name,pdes,a,sigma_a,e,sigma_e,q,sigma_q,ad,sigma_ad,i,sigma_i,om,sigma_om,w,sigma_w,per.y,sigma_per,H,H_sigma,source&sb-kind=a&sb-cdata=%7B%22AND%22:%5B%22a%7CGE%7C150%22,%22q%7CGE%7C50%22%5D%7D'
with urlopen(url) as response:
    ssd = json.load(response)
    #print(ssd)


Objekt für jeden Asteroiden bauen

In [100]:
sednoids = []
for asteroid in ssd['data']:
    # print(asteroid)
    sednoid = {}
    for field, val in zip(ssd['fields'], asteroid):
        sednoid[field] = val
        #print(field, val)
        
    sednoids.append(sednoid)

epoch.cal 2022-01-21.0
full_name  90377 Sedna (2003 VB12)
name Sedna
pdes 90377
a 510.3
sigma_a 1.8e-01
e 0.8504
sigma_e 5.2e-05
q 76.370
sigma_q 1.8e-03
ad 944.33
sigma_ad 3.3e-01
i 11.93
sigma_i 3.7e-06
om 144.21
sigma_om 4.6e-04
w 311.02
sigma_w 3.2e-03
per.y 1.15e+04
sigma_per 2.2e+03
H 1.56
H_sigma None
source ORB
epoch.cal 2022-01-21.0
full_name 541132 Leleakuhonua (2015 TG387)
name Leleakuhonua
pdes 541132
a 1274
sigma_a 1.5e+02
e 0.9489
sigma_e 6.4e-03
q 65.166
sigma_q 2.1e-01
ad 2483.41
sigma_ad 3.0e+02
i 11.66
sigma_i 6.3e-04
om 300.83
sigma_om 7.2e-03
w 117.59
sigma_w 3.1e-01
per.y 4.55e+04
sigma_per 3.0e+06
H 5.50
H_sigma 0.13
source ORB
epoch.cal 2022-01-21.0
full_name        (2012 VP113)
name None
pdes 2012 VP113
a 269.2
sigma_a 5.7e-01
e 0.7012
sigma_e 7.2e-04
q 80.438
sigma_q 3.3e-02
ad 458.02
sigma_ad 9.6e-01
i 24.07
sigma_i 5.8e-04
om 90.75
sigma_om 1.3e-03
w 293.75
sigma_w 1.3e-01
per.y 4.42e+03
sigma_per 5.1e+03
H 4.00
H_sigma None
source ORB
epoch.cal 2022-01-21.0


### Wikitable

Wikitable-Header und -Footer

In [106]:
header = """{| class="wikitable sortable zebra" style="text-align:right; padding:2px 30px 2px 2px;"
|+ Bahnparameter hoch-extremer transneptunischer Objekte mit Perihelien größer 50 AE und großen Halbachsen weiter als 150 AE<ref name="Ssd-Api">
{{Internetquelle 
    |url=https://ssd-api.jpl.nasa.gov/sbdb_query.api?fields=epoch.cal,full_name,name,pdes,a,sigma_a,e,sigma_e,q,sigma_q,ad,sigma_ad,i,sigma_i,om,sigma_om,w,sigma_w,per.y,sigma_per,H,H_sigma,source&sb-kind=a&sb-cdata=%7B%22AND%22:%5B%22a%7CGE%7C150%22,%22q%7CGE%7C50%22%5D%7D 
    |titel=SSD-Daten 
    |werk=[https://ssd.jpl.nasa.gov/tools/sbdb_query.html Small-Body Database Query] 
    |hrsg=Caltech/JPL 
    |format=JSON-Daten 
    |abruf=2022-01-30 
    |abruf-verborgen=1 }}</ref>
|- style="vertical-align:bottom;"
! Objekt
! [[Halbachsen der Ellipse|Halb&shy;achse]]<br />a (AE)
! [[Exzentrizität (Astronomie)|Exzen&shy;trizität]]<br />e
! [[Perihel]]<br />q (AE)
! [[Aphel]]<br />Q (AE)
! [[Bahnneigung|Inkli&shy;nation]]<br />i (°)
! [[Argument der Periapsis|Argument<br />der Periapsis]]<br />(°)
! [[Länge des aufsteigenden Knotens|Länge des aufst. Knotens]]<br />Ω (°)
! [[Umlaufzeit|Umlauf&shy;zeit]]<br />T (Jahre)
! [[Absolute Helligkeit]]<br />H (mag)
"""

footer="""|}<noinclude>

[[Kategorie:Vorlage:Astronomie]]
[[Kategorie:Vorlage:Tabelle]]
</noinclude>
"""

Hilfsfunktion um Toleranzen zu berechen, darzustellen und sie bei zu kleinen Werten auszublenden.

In [107]:
def renderToleran(wert, sigma):
    if sigma * 100 / wert > 2:
        # Toleranz anzeigen
        return "{0:n} ± {1:.0f}".format(wert, sigma)
    else:
        # nur Wert
        return format(wert, "n")

Formatierung Name wikipedia-gerecht (einschließlich Wikilink)

In [108]:
def wikiname(sed):
    if (sed['name'] == None):
        parts = sed['pdes'].split(' ')
        des = parts[1][:2]
        number = parts[1][2:]
        return '[[{}|{} {}<sub>{}</sub>]]'.format(sed['pdes'],parts[0], des, number)
    else:
        return '[[({}) {}]]'.format(sed['pdes'], sed['name'])

Über Asteroiden iterieren und Feldern aus den Asteroiden berechnen

In [109]:
import locale
loc = locale.getlocale()
locale.setlocale(locale.LC_ALL, '')

out = []
for sed in sednoids:
    sed_out = {}
    for name,value in sed.items():
        if name in ['e','q','i','om','w','H']:
            sed_out[name] = format(float(sed[name]), "n")
            #print(name+': '+sed_out[name])
            
        elif name in ['a','ad']:
            sed_out[name] = renderToleran(float(sed[name]), float(sed['sigma_'+name]))
            #print(name+': '+sed_out[name])
            
        elif name == 'per.y':
            # sigma_per sind Tage
            sed_out[name] = renderToleran(float(sed[name]), float(sed['sigma_per'])/365)
            #print(name+': '+sed_out[name])
            
        elif name == 'full_name':
            sed_out[name] = wikiname(sed)
            #print(sed_out[name])
            
        else:
            sed_out[name] = str(sed[name])
            #print('unchanged '+sed_out[name])

    out.append(sed_out)
#print(out)

In [110]:
table = ""
for ast in out:
    #print(ast)
    # Start Zeile
    table += """|-
| style="text-align:left;" | """
    table += ast['full_name'].strip() + '\n'
    table += '| ' + ast['a']
    table += ' || ' + ast['e']
    table += ' || ' + ast['q']
    table += ' || ' + ast['ad']
    table += ' || ' + ast['i']
    table += ' || ' + ast['w']
    table += ' || ' + ast['om']
    table += ' || ' + ast['per.y']
    table += ' || ' + ast['H']

    table += '\n'
    
ergebnis = header + table + footer
print(ergebnis)

{| class="wikitable sortable zebra" style="text-align:right; padding:2px 30px 2px 2px;"
|+ Bahnparameter hoch-extremer transneptunischer Objekte mit Perihelien größer 50 AE und großen Halbachsen weiter als 150 AE<ref name="Ssd-Api">{{Internetquelle |url=https://ssd-api.jpl.nasa.gov/sbdb_query.api?fields=epoch.cal,full_name,name,pdes,a,sigma_a,e,sigma_e,q,sigma_q,ad,sigma_ad,i,sigma_i,om,sigma_om,w,sigma_w,per.y,sigma_per,H,H_sigma,source&sb-kind=a&sb-cdata=%7B%22AND%22:%5B%22a%7CGE%7C150%22,%22q%7CGE%7C50%22%5D%7D |titel=SSD-Daten |werk=[https://ssd.jpl.nasa.gov/tools/sbdb_query.html Small-Body Database Query] |hrsg=Caltech/JPL |format=JSON-Daten |abruf=2022-01-30 |abruf-verborgen=1 }}</ref>
|- style="vertical-align:bottom;"
! Objekt
! [[Halbachsen der Ellipse|Halb&shy;achse]]<br />a (AE)
! [[Exzentrizität (Astronomie)|Exzen&shy;trizität]]<br />e
! [[Perihel]]<br />q (AE)
! [[Aphel]]<br />Q (AE)
! [[Bahnneigung|Inkli&shy;nation]]<br />i (°)
! [[Argument der Periapsis|Argument<br />der 